# 2. Creating a dataframe 
`We create a pandas dataframe to store all the recipes' data

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import regex as re
import itertools
from appel import get_data

We retrieve all the recipes' url, from the search pages :

In [2]:
url, soup, recette, res, marmit = {}, {}, {}, {}, "https://www.marmiton.org/recettes/recherche.aspx?aqt="

url["moelleux"] = marmit + "moelleux-chocolat"
url["micuit"] = marmit + "mi-cuit-chocolat"
url["fondant"] = marmit + "fondant-chocolat"
url["cake"] = marmit + "cake-chocolat"
url["cupcake"] = marmit + "cupcake-chocolat"

def getSubLinks(url):
    links = bs(requests.get(url).content).find_all("a", class_ = "MRTN__sc-1gofnyi-2 gACiYG") # get all pages
    return ['https://www.marmiton.org'+link['href'] for link in links]

for k, u in url.items():
    soup[k] = bs(requests.get(u).content).find_all("div", class_ = "SHRD__sc-1ymbfjb-0 fNHOUq") # get ref block
    recette[k] = [getSubLinks(u)] + [getSubLinks(u+"&page="+p.get_text()) for p in soup[k]] # get all links
    res[k] = list(itertools.chain(*recette[k])) # flatten the list



In [3]:
print(res)

{'moelleux': ['https://www.marmiton.org/recettes/recette_moelleux-au-chocolat_17982.aspx', 'https://www.marmiton.org/recettes/recette_cookies-aux-pepites-de-chocolat-super-moelleux_57330.aspx', ...]}


We create a function get_data() that webscrape each page:

In [4]:
def get_data(key,url):

    soup = bs(requests.get(url).content)

    name = soup.find("h1", class_ = "SHRD__sc-10plygc-0 itJBWW").get_text()
    note = soup.find("span",class_ = "SHRD__sc-10plygc-0 jHwZwD").get_text()

    number_comments = soup.find_all("span",class_ = "SHRD__sc-10plygc-0 cAYPwA")
    number_comments = [nb(string=re.compile("commentaires")) for nb in number_comments]
    number_comments = str([nb for nb in number_comments if nb]).replace('[','').replace(']','').replace(' commentaires','').replace('\'', '')
    

    difficulty = soup.find_all("p",class_ = "RCP__sc-1qnswg8-1 iDYkZP")
    time = soup.find_all("span",class_ = "SHRD__sc-10plygc-0 bzAHrL")
    quantity = soup.find_all("span",class_ = "SHRD__sc-10plygc-0 epviYI")
    ingr = soup.find_all("span", {'class':['RCP__sc-8cqrvd-3 cDbUWZ', 'RCP__sc-8cqrvd-3 itCXhd']})

    #create the list of ingredient, quantity, difficulty, and time
    ingredient = []
    for index,row in enumerate(ingr):
            ingredient.append(row.get_text().replace("\xa0", " ").lower()) 

    quant = []
    for index,row in enumerate(quantity):
            quant.append(row.get_text().replace("\xa0", " ")) 
            
    diff = []
    for index,row in enumerate(difficulty):
        diff.append(row.get_text().replace("\xa0", " ")) 

    ti = []
    for index,row in enumerate(time):
        ti.append(row.get_text().replace("\xa0", " ")) 

    #creation of the DataFrame
    df = pd.DataFrame(quant).transpose()
    df.columns = ingredient

    df['name'] = name
    df['type'] = key
    df['note'] = note
    df['number_comments'] = int(number_comments)

    if int(number_comments)>10:
    #df['url'] = url

        df_diff = pd.DataFrame(diff).transpose()
        df_diff =df_diff.rename(columns={0: "time_prep", 1: "difficulty",2: "price"}).drop("time_prep",axis = 1)

        df_ti = pd.DataFrame(ti).transpose()
        df_ti =df_ti.rename(columns={0: "time_tot", 1: "time_prep",2: "time_repos",3: "time_cooking"})

        frames = [df, df_diff, df_ti]

        result = pd.concat(frames,axis = 1)

        return result
    else:
        print("Not enough comments")

We apply the function on the list of url, to get the database:

In [5]:
recette_data_list = pd.DataFrame()
for key, value in res.items():
    for recipe_url in value:
        try:
            frames = [recette_data_list,get_data(key,recipe_url)] 
            recette_data_list = pd.concat(frames)
        except Exception as e:
            print("Lien n'existe pas :"+ recipe_url)

#les liens qui n'existent pas ont plusieurs ingrédients du même type (chocolat au lait, chocolat noir)

Not enough comments
Lien n'existe pas :https://www.marmiton.org/recettes/recette_moelleux-au-chocolat-coeur-fondant_165276.aspx
Lien n'existe pas :https://www.marmiton.org/recettes/private/recette_pain-d-epices-aux-pepites-de-chocolat_532109.aspx
Lien n'existe pas :https://www.marmiton.org/recettes/recette_gateau-moelleux-chocolat-et-coco_13446.aspx
Not enough comments
Lien n'existe pas :https://www.marmiton.org/recettes/recette_irresistible-moelleux-orange-et-chocolat_24312.aspx
Not enough comments
Lien n'existe pas :https://www.marmiton.org/recettes/recette_cupcake-moelleux-au-chocolat_334835.aspx
Lien n'existe pas :https://www.marmiton.org/recettes/recette_fondant-extra-moelleux-au-chocolat-et-noix-de-coco_40315.aspx
Not enough comments
Lien n'existe pas :https://www.marmiton.org/recettes/recette_moelleux-au-chocolat-avec-glacage-de-vaness_71859.aspx
Lien n'existe pas :https://www.marmiton.org/recettes/recette_gateau-moelleux-chocolat-noix-de-pecan-glacage-chocolat_37937.aspx
Lien n

Let's have a look at the information we get:

In [6]:
display(recette_data_list)

,chocolat,beurre,sucre glace,farine,oeufs,name,type,note,number_comments,difficulty,...,myrtilles,extrait d'amande amère,cardamome,colorant alimentaire,blanc d'oeuf,vermicelles,pépite caramélisée,rose,colorant,oréo
0,250 g,175 g,125 g,75 g,5,Moelleux au chocolat,moelleux,4.8/5,358,très facile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,300 g,250 g,NaN,350 g,2,Cookies aux pépites de chocolat super moelleux,moelleux,4.7/5,715,très facile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,125 g,4,Véritable moelleux au chocolat,moelleux,4.6/5,1677,très facile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,40 g,NaN,200 g,3,Gateau moelleux marbré vanille chocolat,moelleux,4.7/5,358,très facile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,200 g,NaN,5 c.à.s,4,Le Moelleux Chocolat d' Oncle Guillaume,moelleux,4.7/5,159,facile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,125 g,100 g,200 g,3,Mini cakes aux myrtilles et au chocolat blanc ...,cupcake,4.6/5,31,facile,...,300 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,175 g,NaN,175 g,250 g,4,Cupcakes au chocolat,cupcake,4.5/5,73,facile,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,NaN,125 g,30 g,160 g,4,Cupcakes Pépites de Chocolat Glaçage Nutella ...,cupcake,4.8/5,22,facile,...,NaN,NaN,NaN,NaN,NaN,,,,NaN,NaN
0,100 g,200 g,150 g,200 g,3,Mes cupcakes aux pépites de chocolat,cupcake,4.7/5,17,facile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN


In [94]:
print(recette_data_list.shape)
l = recette_data_list.columns

#The different label of sugar:
matches = [match for match in l if "sucre" in match]
print(matches)



(167, 146)
['sucre glace', 'sucre semoule', 'sucre brun', 'sucre en poudre', 'sucre vanillé', 'sucre', 'sucre roux', 'sucre de canne']


In [60]:
for col in recette_data_list:
  print(col)
  print(recette_data_list[col].unique())

chocolat
['250 g' '300 g' nan '125 g' '200 g' '1' '180 g' '30 g' '100 g' '60 g'
 '150 g' '' '50 g' '160 g' '120 g' '1 poignée' '600 g' '175 g' '8 carrés'
 '375 g' ' ' '2' '6 c.à.s' '90 g' '75 g' '40 g' '1 sachet' '1 c.à.s']
beurre
['175 g' '250 g' nan '40 g' '200 g' '60 g' '50 g' '125 g' '150 g' '120 g'
 '70 g' '100 g' '80 g' '75 g' '180 g' '30 g' '1 c.à.s' '' '72.5 g' '65 g'
 '140 g' '20 g' '10 g' '225 g' '90 g' '190 g' '160 g' '1 noix' '110 g'
 '130 g' '37 g' '45 g' '3 cuillères' '85 g' '1 noisette' '170 g'
 '1⁄2 verre' '2 c.à.s' '25 g' '185 g' '165 g' '300 g' '4 boîtes']
sucre glace
['125 g' nan '40 g' '2 c.à.s' '' '80 g' '250 g' '175 g' '160 g' '50 g'
 '100 g' '75 g' '20 g' '200 g' '150 g' '3 c.à.s' '30 g' '180 g' '70 g']
farine
['75 g' '350 g' '125 g' '200 g' '5 c.à.s' '80 g' '100 g' nan '60 g'
 '1 t.à.c' '120 g' '2 pots' '1 c.à.s' '275 g' '30 g' '45 g' '150 g'
 '180 g' '50 g' '40 g' '240 g' '300 g' '70 g' '250 g' '10 g' '225 g'
 '25 g' '2 c.à.s' '3 c.à.s' '55 g' '4 c.à.s' '65 g' 

Next step: Data cleaning !